### Data and parameter choices explained
Looking at SR 101 in the Phoenix area. This is a 62 miles beltway connecting bedroom communities in the Phoenix metro area. The most congested portion is between Exit 53 Broadway and Apache Blvd (128,692 cars / 0.19 miles, one direction).

Some data sources 
- https://azdot.gov/planning/data-and-information/traffic-monitoring
- https://azdot.gov/sites/default/files/2025-06/2024-AADT-PUBLICATION_StateRoutes.pdf
- Data dictionary: https://azdot.gov/sites/default/files/2019/08/data-dictionary.pdf 
- External assessment of busyness: https://www.access-insurance.com/traffic-patterns-phoenix-arizona
- Project description: https://planningtools.transportation.org/290/view-case-study.html?case_id=57 

Consider I-17 (part of the route connecting Phoenix and Flagstaff)
- Consider segments 17-6 through 17-12 (total length of 77 miles)
- A lane is 12 feet wide. All segments have 4 lanes except for 17-7 which has 5 lanes (length of 9 miles)
- Most of these road segments are rural in nature

Here is the main data of interest:

Segment: Length || # Lanes || Weighted Avg Speed Limit || AADT (2019) || Daily Capacity
- 17-6: 16mi (25.74944 km) || 4 || 75 || 28,900 || 69,309
- 17-7: 9mi (14.48406 km) || 5 || 65 || 33,600 || 70,445
- 17-8: 11mi (17.70274 km) || 4 || 75 || 28,500 || 75,721
- 17-9: 8mi (12.87472 km) || 4 || 75 || 21,600 || 55,092
- 17-10: 9mi (14.48406 km) || 4 || 75 || 18,200 || 56,356
- 17-11: 7mi (11.26538 km) || 4 || 75 || 22,200 || 69,309
- 17-12: 17mi (27.35878 km) || 4 || 71 || 26,800 || 79,947

I assume AADT and Daily Capacity are for both directions?

So what if we smooth this over? The weighted average

Source: https://azdot.gov/sites/default/files/media/2022/10/I-17_Final_Report_w_Appendices_06-30-22.pdf

In [2]:
import pandas as pd
import numpy as np

In [10]:
segments = ["17-6", "17-7", "17-8", "17-9", "17-10", "17-11", "17-12"]
lengths_mi = [16, 9, 11, 8, 9, 7, 17] # in miles
lengths_km = np.array(lengths_mi) * 1.60934
lanes = [4, 5, 4, 4, 4, 4, 4] # each lane is 12 feet wide
wt_avg_speed_limit_mph = [75, 65, 75, 75, 75, 75, 71]
wt_avg_speed_limit_kmh = np.array(wt_avg_speed_limit_mph) * 1.60934
aadt = [28900, 33600, 28500, 21600, 18200, 22200, 26800]
daily_cap = [69309, 70445, 75721, 55092, 56356, 69309, 79947]
data_dict = {"segment": segments, "length (mi)": lengths_mi, "length (km)": lengths_km, "lanes": lanes, "wt avg speed limit (mph)": wt_avg_speed_limit_mph, "wt avg speed limit (kmh)": wt_avg_speed_limit_kmh, "aadt": aadt, "daily capacity": daily_cap}
data = pd.DataFrame(data_dict)
data.set_index("segment", inplace = True, drop = True)
data["hourly capacity"] = data["daily capacity"] / 24
data["capacity per km"] = data["hourly capacity"] / data["wt avg speed limit (kmh)"]
data

,length (mi),length (km),lanes,wt avg speed limit (mph),wt avg speed limit (kmh),aadt,daily capacity,hourly capacity,capacity per km
segment,,,,,,,,,
17-6,16,25.74944,4,75,120.70050,28900,69309,2887.875000,23.925957
17-7,9,14.48406,5,65,104.60710,33600,70445,2935.208333,28.059361
17-8,11,17.70274,4,75,120.70050,28500,75721,3155.041667,26.139425
17-9,8,12.87472,4,75,120.70050,21600,55092,2295.500000,19.018148
17-10,9,14.48406,4,75,120.70050,18200,56356,2348.166667,19.454490
17-11,7,11.26538,4,75,120.70050,22200,69309,2887.875000,23.925957
17-12,17,27.35878,4,71,114.26314,26800,79947,3331.125000,29.153102


What if this was just a single stretch of 77 miles (123.9 km) with the same characteristics everywhere?

In [13]:
# Total length (km)
total_length_km = sum(data["length (km)"])
print("Total length (km):", total_length_km)

# Weighted average number of lanes
wt_avg_lanes = sum(data["length (km)"] * data["lanes"]) / sum(data["length (km)"])
print("# Lanes (weighted average):", wt_avg_lanes)

# Weighted average speed limit
wt_avg_speed_limit_kmh = sum(data["length (km)"] * data["wt avg speed limit (kmh)"]) / sum(data["length (km)"])
print("Speed limit (weighted average):", wt_avg_speed_limit_kmh)

# Weighted average aadt
wt_avg_speed_limit_kmh = sum(data["length (km)"] * data["aadt"]) / sum(data["length (km)"])
print("AADT (weighted average):", wt_avg_speed_limit_kmh)

# Weighted average daily capacity
wt_avg_cap = sum(data["length (km)"] * data["daily capacity"]) / sum(data["length (km)"])
print("Daily capacity (weighted average): ", wt_avg_cap)
wt_avg_cap_hrly = sum(data["length (km)"] * data["hourly capacity"]) / sum(data["length (km)"])
print("Hourly capacity (weighted average): ", wt_avg_cap_hrly)

# Weighted average capacity / km
wt_avg_cap_km = sum(data["length (km)"] * data["capacity per km"]) / sum(data["length (km)"])
print("Cars / km (weighted average): ", wt_avg_cap_km)

# Suppose those cars are traveling at the (weighted avg) posted speed limit?

Total length (km): 123.91917999999998
# Lanes (weighted average): 4.116883116883117
Speed limit (weighted average): 117.39821792207793
AADT (weighted average): 26310.389610389615
Daily capacity (weighted average):  69715.35064935066
Hourly capacity (weighted average):  2904.806277056277
Cars / km (weighted average):  24.84679570325913


In [6]:
sum(data["hourly capacity"]) / sum(data["length (km)"])

160.11074045734216

In [25]:
((16*69309) + (9*70445) + (11*75721) + (8*55092) + (9*56356) + (7*69309) + (17*79947)) / 77

69715.35064935064

In [40]:
(69715/123.9)/24


23.444646220069945

In [41]:
69715 /(24*117.4)

24.74268881317433